# CityShadows: A Shade-based Route Recommendation Tool

This notebook loads building footprints with heights, and tree data from an area in Makati, Metro Manila. It takes a pair of start-end coordinates and recommends a route with the best amount of shade along it given a date and time.

### Requirements:
- **Python 3.9** in order for pybdshadow to function properly.
- .env file containing an **Openrouteservice** (https://openrouteservice.org/) API key (OSM_API)

## Libraries

In [ ]:
from dotenv import load_dotenv
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polyline
import openrouteservice
import pybdshadow
from shapely.geometry import LineString
from shapely.ops import unary_union
from shapely import LineString
import os
import matplotlib.patheffects as pe

## Variables

### shp files

In [ ]:
small_makati = gpd.read_file("../data/smaller_makati.shp")
buildings = gpd.read_file("../data/gis_osm_buildings_a_free_1.shp")
less_makati_buildings = buildings[small_makati['geometry'].item().contains(buildings['geometry'])]
roads = gpd.read_file("../data/gis_osm_roads_free_1.shp")
less_makati_roads = roads[small_makati['geometry'].item().contains(roads['geometry'])]

### geojson files (buildings + trees)

In [ ]:
buildings_dataset = gpd.read_file("mapping_heights/buildings_dataset.geojson")
trees_dataset = gpd.read_file("mapping_heights/trees_dataset.geojson")
tree_clusters = gpd.read_file("mapping_heights/tree_clusters.geojson")

### api

In [ ]:
load_dotenv(override=True)
OSM_API = os.getenv('OSM_API')

### start & end coordinates

In [ ]:
start_lat, start_long = 121.01035385542158, 14.557826176363248
end_lat, end_long = 121.01690365415651, 14.55576083029497

### date & time

In [ ]:
date_time_input = pd.to_datetime('2025-08-08 09:00:00') # year-month-day hour:minute:second

## Function Declaration

### Helpers

In [ ]:
def convert_ph_to_utc(ph_datetime):
    """
    Converts a datetime in Philippine time to UTC.
    
    Parameters:
      ph_datetime (str or datetime-like): A datetime in Philippine time.
    
    Returns:
      A pandas.Timestamp representing the time in UTC.
    """
    ts = pd.to_datetime(ph_datetime)
    ts_utc = ts.tz_localize('Asia/Manila').tz_convert('UTC')
    return ts_utc

### Routing

In [ ]:
def getOSMRoutes(start_lat, start_long, end_lat, end_long, apiKey):
    """
    Retrieves up to 6 alternative walking routes between two coordinates using the OpenRouteService API.

    The function makes two routing requests:
    1. With default ORS alternative route settings (weight_factor=1.4, share_factor=0.6).
    2. With custom settings (weight_factor=2, share_factor=0.2, preference='shortest').

    It deduplicates the results based on the route geometry to ensure only unique routes are returned.

    Parameters:
        start_lat (float): Latitude of the start location.
        start_long (float): Longitude of the start location.
        end_lat (float): Latitude of the end location.
        end_long (float): Longitude of the end location.
        apiKey (str): OpenRouteService API key.

    Returns:
        list of dict: A list of route dictionaries, each containing:
            - 'coordinates': list of (lat, lon) tuples representing the route path.
    """
    client = openrouteservice.Client(key=apiKey)

    # 1st Request: Default ORS settings
    requestParams1 = {'coordinates': [[start_lat, start_long],
                                       [end_lat, end_long]],
                      'profile': 'foot-walking',
                      'alternative_routes': {
                          'target_count': 3,
                          'weight_factor': 1.4,
                          'share_factor': 0.6
                      }}
    # 2nd Request: Custom ORS settings 
    requestParams2 = {'coordinates': [[start_lat, start_long],
                                       [end_lat, end_long]],
                      'profile': 'foot-walking',
                      'alternative_routes': {
                          'target_count': 3,
                          'weight_factor': 2,
                          'share_factor': 0.2
                      }, 
                      'preference': 'shortest'
                      }

    routeRequest1  = client.directions(**requestParams1)
    routeRequest2  = client.directions(**requestParams2)

    # Set to store unique geometry strings (routes) only
    unique_geometries = set()
    routeData = []
    routeDistance=[]

    for routeRequest in [routeRequest1, routeRequest2]:
        for route in routeRequest.get("routes", []):
            geometry_str = route["geometry"]  
            if geometry_str not in unique_geometries: 
                unique_geometries.add(geometry_str)
                routeData.append(polyline.decode(geometry_str))
                routeDistance.append(route['summary']['distance'])
    return routeData,routeDistance


def convert_routes_to_linestrings(routes):
    """
    Converts a list of route dictionaries into Shapely LineString objects.

    This function takes decoded route coordinates (in (lat, lon) format)
    and transforms them into LineString geometries (in (lon, lat) format)
    for spatial analysis and GIS compatibility.

    Parameters:
        routes (list of dict): A list of route dictionaries with a key "coordinates"
            containing a list of (lat, lon) tuples.

    Returns:
        list of shapely.geometry.LineString: A list of LineString objects representing the routes.
    """

    linestrings = []
    for route in routes:
        coords = LineString([(lon, lat) for (lat, lon) in route])
        linestrings.append(coords)
    return linestrings

### Detecting nearby buildings

In [ ]:
def extractStructuresNearRoad(road, structureDF, threshold):
    """
    Returns structures within the specified distance threshold from any road LineString.
    
    Parameters:
        road (list of LineStrings): List of road geometries.
        structureDF (GeoDataFrame): GeoDataFrame of structures (must have geometry column).
        threshold (float): Distance threshold in degrees (e.g., 0.0005 ≈ 55 meters).
    
    Returns:
        GeoDataFrame: Structures near any of the road lines.
    """
    # Use apply to check distance from each building to any of the LineStrings
    nearby_structures = structureDF[structureDF.geometry.apply(
        lambda geom: any(r.distance(geom) <= threshold for r in road)
    )]
    return nearby_structures


def plotBuildingNearRoad(road, buildingDF, adjacentBuildingsDF):
    """
    Plots all buildings, highlights adjacent buildings in red, and road routes in blue.
    
    Parameters:
        road (list of LineStrings): List of route geometries.
        buildingDF (GeoDataFrame): All buildings.
        adjacentBuildingsDF (GeoDataFrame): Filtered buildings near road.
    """
    # Convert input to GeoDataFrame if needed
    adjacent_buildings = gpd.GeoDataFrame(adjacentBuildingsDF, crs="EPSG:4326")
    adjacent_buildings.set_geometry('geometry', inplace=True)
    
    # Combine road geometries into a GeoSeries for plotting
    sample_road = gpd.GeoSeries(road, crs="EPSG:4326")

    # Plot
    ax = buildingDF.plot(color='lightgray', edgecolor='black', figsize=(10, 10))
    adjacent_buildings.plot(ax=ax, color='red', label='Adjacent Buildings')
    sample_road.plot(ax=ax, color='blue', label='Road')
    
    plt.legend()
    plt.title("Buildings Near Road")
    plt.show()

### Shadow calculation

In [ ]:
def tryGeneratingShadow(structures, date):
    # Define a flag that forces the area to be marked as shaded if the time is outside daylight.
    force_shade = True
    is_daylight = True      # To determine if it is before sunrise or after sunset

    # Try to calculate shadows. If the time is before sunrise or after sunset, handle based on force_shade.
    try:
        shadows = pybdshadow.bdshadow_sunlight(
            structures, 
            date, 
            height='height',      
            roof=False, 
            include_building=True, 
            ground=0
        )
    except ValueError as e:
        if "before sunrise or after sunset" in str(e) and force_shade:
            print("Time is outside daylight hours. Marking entire area as shaded (force_shade=True).")
            shadows = structures.copy()  # Entire area is treated as shaded.
            is_daylight = False
        else:
            raise
    return shadows, is_daylight


def compute_shadow_percentage(route, shadows_gdf):
    """
    Computes the percentage of a route that lies in shadow.

    Parameters:
      route (shapely.geometry.LineString): The route geometry in EPSG:4326.
      shadows_gdf (GeoDataFrame): A GeoDataFrame of shadow geometries (should be EPSG:4326 or have a CRS).

    Returns:
      float: Percentage of the route length that is shaded (0.0–100.0).
    """
    # Project the route into meters
    route_proj = (
        gpd.GeoSeries([route], crs="EPSG:4326")
           .to_crs(epsg=3857)
           .iloc[0]
    )

    # Ensure shadows have a CRS, then project
    if shadows_gdf.crs is None:
        shadows_gdf = shadows_gdf.set_crs("EPSG:4326")
    shadows_proj = shadows_gdf.to_crs(epsg=3857).copy()

    # Repair invalid geometries by buffering zero
    shadows_proj["geometry"] = shadows_proj.geometry.buffer(0)

    # Drop any empty or still-invalid geometries
    valid_geoms = [
        geom for geom in shadows_proj.geometry
        if geom is not None and not geom.is_empty and geom.is_valid
    ]

    # Union all the valid shadow polygons
    union_shadow = unary_union(valid_geoms)

    # Intersect with the route and measure shaded length
    inter = route_proj.intersection(union_shadow)
    shaded_length = inter.length if not inter.is_empty else 0.0

    # Compute percentage
    total_length = route_proj.length
    if total_length == 0:
        return 0.0

    return (shaded_length / total_length) * 100.0

## Getting the routes
This section generates 3-6 routes from start_lat, start_long to end_lat, end_long. This is done using OpenRouteService with two different routing configurations. The routes are saved as LineString objects for GIS operations.

In [ ]:
routes,distances = getOSMRoutes(start_lat, start_long, end_lat, end_long, OSM_API)
routes = convert_routes_to_linestrings(routes)
routes

Each linestring will be converted into a polygon to alter its thickness.

In [ ]:
routes_gdf = gpd.GeoDataFrame(geometry=routes, crs="EPSG:4326")
routes_utm = routes_gdf.to_crs(epsg=32651)

routes_buffered = routes_utm.copy()
routes_buffered["geometry"] = routes_utm.geometry.buffer(5)

routes_buffered = routes_buffered.to_crs(epsg=4326)
print(routes_buffered)

## Finding nearest buildings from route/s whose shadows will be considered
This sections gets the nearest buildings from all routes whose shadows will be considered. A higher threshold includes more buildings and vice versa.

In [ ]:
buildings_gdf4326 = buildings_dataset.to_crs("EPSG:4326")
trees_gdf4326     = trees_dataset.to_crs("EPSG:4326")
tree_clusters_gdf4326 = tree_clusters.to_crs("EPSG:4326")

adjacent_buildings = extractStructuresNearRoad(road = routes, 
                                               structureDF = buildings_gdf4326, 
                                               threshold = 0.0005)

adjacent_trees = extractStructuresNearRoad( road = routes,
                                            structureDF = trees_gdf4326,
                                            threshold   = 0.0005)         

adjacent_tree_clusters = extractStructuresNearRoad( road = routes,
                                            structureDF = tree_clusters_gdf4326,
                                            threshold   = 0.0005)                 

plotBuildingNearRoad(road = routes, 
                     buildingDF = buildings_gdf4326,
                     adjacentBuildingsDF = adjacent_buildings)


plotBuildingNearRoad(road = routes, 
                     buildingDF = trees_gdf4326,
                     adjacentBuildingsDF = adjacent_trees)

plotBuildingNearRoad(road = routes, 
                     buildingDF = tree_clusters_gdf4326,
                     adjacentBuildingsDF = adjacent_tree_clusters)

## Shadow calculation of buildings and trees

In [ ]:
processed_buildings = pybdshadow.bd_preprocess(adjacent_buildings, height='height')
processed_trees = pybdshadow.bd_preprocess(adjacent_trees, height='height')

In [ ]:
date = pd.to_datetime(convert_ph_to_utc(date_time_input))  

shadows_buildings, is_daylight = tryGeneratingShadow(processed_buildings, date)
shadows_trees, is_daylight = tryGeneratingShadow(processed_trees, date)

In [ ]:
shadows_buildings, is_daylight = tryGeneratingShadow(processed_buildings, date)
shadows_trees, is_daylight = tryGeneratingShadow(processed_trees, date)

routes_gdf= gpd.GeoDataFrame(geometry=routes, crs="EPSG:4326")

# Plot
fig, ax = plt.subplots(figsize=(10, 10))
processed_buildings.plot(ax=ax, color='blue', alpha=0.7, label='Buildings')
shadows_buildings.plot(ax=ax, color='gray', alpha=0.5, edgecolor='black', label='Shadows')
processed_trees.plot(ax=ax, color='green', alpha=0.7, label='Trees')
shadows_trees.plot(ax=ax, color='gray', alpha=0.5, edgecolor='black', label='Shadows')
adjacent_tree_clusters.plot(ax=ax, color='gray', alpha=0.5, label='Shadows')

routes_gdf.plot(ax=ax, color='red', linewidth=2, label='Route(s)')
plt.legend()
plt.title("Building and Tree Shadows")
plt.show()

## Route recommendation

In [ ]:
NATIVE_CRS = "EPSG:4326"

for name, gdf in [
    ("shadows_buildings", shadows_buildings),
    ("shadows_trees",    shadows_trees),
    ("tree_clusters",    tree_clusters),
    ("processed_trees",  processed_trees),
    ("routes_buffered",  routes_buffered),
]:
    if gdf.crs is None:
        gdf.set_crs(NATIVE_CRS, inplace=True)
        print(f"Assigned CRS {NATIVE_CRS} to {name}")

crs_target = shadows_buildings.crs

sb = shadows_buildings.to_crs(crs_target)
st = shadows_trees.to_crs(crs_target)
tc = tree_clusters[['geometry']].to_crs(crs_target)
pt = processed_trees[['geometry']].to_crs(crs_target)

shaded_areas = gpd.GeoDataFrame(
    pd.concat([sb, st, tc, pt], ignore_index=True),
    crs=crs_target
)

# repair any invalid polygons in shaded_areas
shaded_areas['geometry'] = shaded_areas.geometry.buffer(0)
shaded_areas = shaded_areas[shaded_areas.is_valid]

In [ ]:
# Compute and print shaded percentage for each route
percentages = []
print("Shaded percentage per route:")
for idx, buf_poly in enumerate(routes_buffered.geometry):
    centerline = buf_poly.boundary  
    prc = compute_shadow_percentage(centerline, shaded_areas)
    percentages.append(prc)
    print(f"  Route {idx}: {prc:.2f}, length={distances[idx]} meters")

# Tie‐breaker logic to pick the best route
best_idx = percentages.index(max(percentages))
lengths = routes_gdf.geometry.length
dupes = [(i, p, lengths[i]) for i, p in enumerate(percentages) if p == percentages[best_idx]]
if len(dupes) > 1:
    # choose shortest among the tied routes
    best_idx = dupes[np.argmin([d[2] for d in dupes])][0]

best_dist = percentages[best_idx]
print(f"\nBest route is #{best_idx}: {best_dist:.2f}% shaded, length={distances[best_idx] * percentages[best_idx] / 100}")

## Route Visualizations

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# plot all the layers
routes_gdf.plot(ax=ax, color='lightgray', linewidth=1, label='Other Routes')
routes_gdf.iloc[[best_idx]].plot(
    ax=ax, color='red', linewidth=3,
    label=f'Chosen Route ({percentages[best_idx]:.1f}% shaded)'
)
processed_buildings.plot(ax=ax, color='blue', alpha=0.7, label='Buildings')
shadows_buildings.plot(ax=ax, color='gray', alpha=0.5, label='Building Shadows')
adjacent_tree_clusters.plot(ax=ax, color='gray', alpha=0.5, label='Tree Cluster Shadows')
routes_buffered.plot(ax=ax, color='lightblue', alpha=0.3, label='Routes')
processed_trees.plot(ax=ax, color='darkgreen', alpha=0.7, label='Trees')
shadows_trees.plot(ax=ax, color='lightgreen', alpha=0.5, label='Tree Shadows')

for idx, row in routes_gdf.iterrows():
    x, y = row.geometry.representative_point().coords[0]
    txt = f"{idx}: {percentages[idx]:.1f}%"
    color = 'red' if idx == best_idx else 'black'
    ax.text(
        x, y, txt,
        ha='center', va='center',
        fontsize=10, fontweight='bold',
        color=color,
        path_effects=[
            pe.Stroke(linewidth=3, foreground='white'),
            pe.Normal()
        ]
    )

ax.legend()
ax.set_title("Best Route with Both Building & Tree Shading")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

routes_gdf.plot(ax=ax, color='lightgray', linewidth=1, label='Other Routes')

routes_gdf.iloc[[best_idx]].plot(
    ax=ax, color='red', linewidth=3,
    label=f'Chosen Route ({best_dist:.1f} distance experienced.)'
)

processed_buildings.plot(ax=ax, color='blue', alpha=0.7, label='Buildings')
shadows_buildings.plot(ax=ax, color='gray', alpha=0.5, label='Building Shadows')
adjacent_tree_clusters.plot(ax=ax, color='gray', alpha=0.5, label='Tree Cluster Shadows')
routes_buffered.plot(ax=ax, color='lightblue', alpha=0.3, label='Routes')

processed_trees.plot(ax=ax, color='darkgreen', alpha=0.7, label='Trees')
shadows_trees.plot(ax=ax, color='lightgreen', alpha=0.5, label='Tree Shadows')

ax.legend()
ax.set_title("Best Route with Both Building & Tree Shading")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.tight_layout()
plt.show()